In [1]:

import pandas as pd


providers = pd.read_csv("/content/providers_data.csv")
receivers = pd.read_csv("/content/receivers_data.csv")
food_listings = pd.read_csv("/content/food_listings_data.csv")
claims = pd.read_csv("/content/claims_data.csv")


print("Providers:", providers.shape)
print("Receivers:", receivers.shape)
print("Food Listings:", food_listings.shape)
print("Claims:", claims.shape)

print("\nProviders Data Sample:")
print(providers.head())

print("\nReceivers Data Sample:")
print(receivers.head())

print("\nFood Listings Data Sample:")
print(food_listings.head())

print("\nClaims Data Sample:")
print(claims.head())


Providers: (1000, 6)
Receivers: (1000, 5)
Food Listings: (1000, 9)
Claims: (1000, 5)

Providers Data Sample:
   Provider_ID                         Name           Type  \
0            1             Gonzales-Cochran    Supermarket   
1            2  Nielsen, Johnson and Fuller  Grocery Store   
2            3                 Miller-Black    Supermarket   
3            4   Clark, Prince and Williams  Grocery Store   
4            5               Coleman-Farley  Grocery Store   

                                             Address            City  \
0  74347 Christopher Extensions\nAndreamouth, OK ...     New Jessica   
1           91228 Hanson Stream\nWelchtown, OR 27136     East Sheena   
2  561 Martinez Point Suite 507\nGuzmanchester, W...  Lake Jesusview   
3     467 Bell Trail Suite 409\nPort Jesus, IA 61188     Mendezmouth   
4  078 Matthew Creek Apt. 319\nSaraborough, MA 53978   Valentineside   

                Contact  
0       +1-600-220-0480  
1  +1-925-283-8901x6297  
2      

In [2]:


import sqlite3


conn = sqlite3.connect("food_wastage.db")
cursor = conn.cursor()


providers.to_sql("providers", conn, if_exists="replace", index=False)
receivers.to_sql("receivers", conn, if_exists="replace", index=False)
food_listings.to_sql("food_listings", conn, if_exists="replace", index=False)
claims.to_sql("claims", conn, if_exists="replace", index=False)

tables = cursor.execute("SELECT name FROM sqlite_master WHERE type='table';").fetchall()
print("Tables in database:", tables)


Tables in database: [('providers',), ('receivers',), ('food_listings',), ('claims',)]


In [4]:

query1 = """
SELECT p.City AS city,
       COUNT(DISTINCT p.Provider_ID) AS total_providers,
       COUNT(DISTINCT r.Receiver_ID) AS total_receivers
FROM providers p
LEFT JOIN receivers r ON p.City = r.City
GROUP BY p.City
ORDER BY p.City;
"""
pd.read_sql_query(query1, conn)


,city,total_providers,total_receivers
0,Adambury,1,0
1,Adamsview,1,0
2,Adamsville,1,0
3,Aguirreville,1,0
4,Alexanderchester,1,0
...,...,...,...
958,Wrightville,1,0
959,Yatesside,1,0
960,Youngchester,1,0
961,Zimmermanton,1,0


In [5]:
query2 = """
SELECT Provider_Type,
       SUM(Quantity) AS total_quantity
FROM food_listings
GROUP BY Provider_Type
ORDER BY total_quantity DESC
LIMIT 1;
"""
pd.read_sql_query(query2, conn)


,Provider_Type,total_quantity
0,Restaurant,6923


In [6]:
city_name = "New Jessica"

query3 = f"""
SELECT Name, Contact, Address
FROM providers
WHERE City = '{city_name}';
"""
pd.read_sql_query(query3, conn)


,Name,Contact,Address
0,Gonzales-Cochran,+1-600-220-0480,"74347 Christopher Extensions\nAndreamouth, OK ..."


In [7]:
query4 = """
SELECT r.Name, COUNT(c.Claim_ID) AS total_claims
FROM receivers r
JOIN claims c ON r.Receiver_ID = c.Receiver_ID
GROUP BY r.Name
ORDER BY total_claims DESC
LIMIT 10;
"""
pd.read_sql_query(query4, conn)


,Name,total_claims
0,William Frederick,5
1,Scott Hunter,5
2,Matthew Webb,5
3,Anthony Garcia,5
4,Kristine Martin,4
5,Kristina Simpson,4
6,Jennifer Nelson,4
7,Donald Caldwell,4
8,Chelsea Powell,4
9,Betty Reid,4


In [8]:
query5 = """
SELECT SUM(Quantity) AS total_food_quantity
FROM food_listings;
"""
pd.read_sql_query(query5, conn)


,total_food_quantity
0,25794


In [9]:
query6 = """
SELECT Location AS city, COUNT(*) AS total_listings
FROM food_listings
GROUP BY Location
ORDER BY total_listings DESC
LIMIT 1;
"""
pd.read_sql_query(query6, conn)


,city,total_listings
0,South Kathryn,6


In [10]:
query7 = """
SELECT Food_Type, COUNT(*) AS count
FROM food_listings
GROUP BY Food_Type
ORDER BY count DESC;
"""
pd.read_sql_query(query7, conn)


,Food_Type,count
0,Vegetarian,336
1,Vegan,334
2,Non-Vegetarian,330


In [11]:
query8 = """
SELECT fl.Food_Name, COUNT(c.Claim_ID) AS total_claims
FROM food_listings fl
LEFT JOIN claims c ON fl.Food_ID = c.Food_ID
GROUP BY fl.Food_Name
ORDER BY total_claims DESC;
"""
pd.read_sql_query(query8, conn)


,Food_Name,total_claims
0,Rice,122
1,Soup,114
2,Dairy,110
3,Fish,108
4,Salad,106
5,Chicken,102
6,Bread,94
7,Pasta,87
8,Vegetables,86
9,Fruits,71


In [12]:
query9 = """
SELECT c.Claim_ID, fl.Food_Name, r.Name AS receiver_name, c.Status
FROM claims c
JOIN food_listings fl ON c.Food_ID = fl.Food_ID
JOIN receivers r ON c.Receiver_ID = r.Receiver_ID
WHERE c.Status = 'Pending';
"""
pd.read_sql_query(query9, conn)


,Claim_ID,Food_Name,receiver_name,Status
0,1,Dairy,Abigail Crawford,Pending
1,5,Pasta,Tina Watkins,Pending
2,8,Salad,Hannah Reynolds,Pending
3,11,Dairy,Matthew Tran,Pending
4,13,Vegetables,Samantha Gomez,Pending
...,...,...,...,...
320,980,Salad,Sandy Ward,Pending
321,983,Pasta,Eugene Beltran,Pending
322,988,Vegetables,Brent Anderson,Pending
323,989,Chicken,Walter Carter,Pending


In [13]:
query10 = """
SELECT Meal_Type, AVG(Quantity) AS avg_quantity
FROM food_listings
GROUP BY Meal_Type;
"""
pd.read_sql_query(query10, conn)


,Meal_Type,avg_quantity
0,Breakfast,26.248031
1,Dinner,24.738776
2,Lunch,25.697581
3,Snacks,26.454545


In [14]:
query11 = """
SELECT p.Provider_ID, p.Name
FROM providers p
LEFT JOIN food_listings fl ON p.Provider_ID = fl.Provider_ID
WHERE fl.Food_ID IS NULL;
"""
pd.read_sql_query(query11, conn)


,Provider_ID,Name
0,3,Miller-Black
1,6,Lawson-Walters
2,8,Curtis-Lewis
3,12,Miller Ltd
4,13,"Parker, Flores and Jennings"
...,...,...
358,988,"Morris, Serrano and Roberson"
359,991,Spencer LLC
360,992,Miller-Sanders
361,994,"Porter, Hernandez and Pace"


In [15]:
query12 = """
SELECT r.Receiver_ID, r.Name
FROM receivers r
LEFT JOIN claims c ON r.Receiver_ID = c.Receiver_ID
WHERE c.Claim_ID IS NULL;
"""
pd.read_sql_query(query12, conn)


,Receiver_ID,Name
0,5,John Romero
1,8,James Perez
2,11,Amber Pacheco
3,13,Jamie Sutton
4,17,Melissa Castillo
...,...,...
371,987,Becky Smith
372,989,Timothy Bradshaw
373,991,Lindsey Wagner
374,992,James Curry


In [16]:
query13 = """
SELECT fl.Meal_Type, COUNT(c.Claim_ID) AS total_claims
FROM food_listings fl
JOIN claims c ON fl.Food_ID = c.Food_ID
GROUP BY fl.Meal_Type
ORDER BY total_claims DESC
LIMIT 1;
"""
pd.read_sql_query(query13, conn)


,Meal_Type,total_claims
0,Breakfast,278


In [17]:
query14 = """
SELECT
    (SELECT SUM(Quantity) FROM food_listings) AS total_available,
    COUNT(c.Claim_ID) AS total_claimed
FROM claims c;
"""
pd.read_sql_query(query14, conn)


,total_available,total_claimed
0,25794,1000


In [18]:
query15 = """
SELECT DISTINCT p.City
FROM providers p
JOIN receivers r ON p.City = r.City;
"""
pd.read_sql_query(query15, conn)


,City
0,New Jessica
1,North Michelle
2,New Daniel
3,Christinaland
4,Huntermouth
5,Perezport
6,Davidport
7,East John
8,Paulmouth
9,Jamesview
